## Initalize notebook

In [1]:
import typing as T

import pandas as pd
from itertools import combinations
import random
import time
from collections import Counter

## Load data

In [2]:
# load projections and salary data
df_projections = pd.read_csv("fantasy_pros_data_week7.csv")
df_salary = pd.read_csv("historical_and_salary_data_week7.csv")

# only keep a few projection columns
df_projections_merge = df_projections[
    ["name_position_key", "start_sit_score", "start_sit_grade", "projected_points"]
]

# merge data
df_players = pd.merge(
    left=df_salary, right=df_projections_merge, how="left", on="name_position_key"
)

# fill NaN with 0
df_players["start_sit_score"] = df_players["start_sit_score"].fillna(0)
df_players["projected_points"] = df_players["projected_points"].fillna(0)
df_players["projected_points_per_salary"] = df_players["projected_points"] / df_players["salary"] * 1000

df_players.head()

,gid,position,name,team,opponent,home/away,salary,salary_change,total_points,games_played,points_per_game,points_per_game_per_salary,points_per_game_alt,bye_week,ytd_salary_high/low,name_position_key,start_sit_score,start_sit_grade,projected_points,projected_points_per_salary
0,1412,QB,"Wilson, Russell",sea,ari,A,8000,0,159.38,5,31.88,3.98,31.88,6,H,russell-wilson-qb,4.33,A+,23.3,2.912500
1,1501,QB,"Prescott, Dak",dal,was,A,4000,0,151.64,5,30.33,7.58,30.33,10,L,dak-prescott-qb,0.00,NaN,0.0,0.000000
2,5562,RB,"Kamara, Alvin",no,car,H,7900,0,150.60,5,30.12,3.81,30.12,6,NaN,alvin-kamara-rb,4.33,A+,23.2,2.936709
3,1523,QB,"Mahomes II, Patrick",kan,den,A,7400,-400,168.46,6,28.08,3.79,28.08,10,L,patrick-mahomes-qb,4.33,A+,24.0,3.243243
4,1537,QB,"Murray, Kyler",ari,sea,H,7100,-200,168.48,6,28.08,3.95,28.08,8,NaN,kyler-murray-qb,4.00,A,22.5,3.169014


In [3]:
# load schedule data
df_schedule = pd.read_csv("schedule_and_odds_week7.csv")
df_schedule["datetime"] = pd.to_datetime(
    df_schedule["datetime"], infer_datetime_format=True
)
df_schedule["hour"] = df_schedule["datetime"].dt.hour

# find valid games in the schedule (Sunday 1pm and 4pm games only)
df_schedule_valid = df_schedule[df_schedule["hour"].isin([1, 4])]
valid_teams = (
    df_schedule_valid["favorite"].unique().tolist()
    + df_schedule_valid["underdog"].unique().tolist()
)

print("number of teams playing on sunday at 1pm or 4pm:", len(valid_teams))

# create a mapping of expected points per team based on the over/under and line
# note the line is negative for the favored team, so we subtract it from the 
#  favorite and add it to the underdog
team_points_mapping = {}
for row in df_schedule_valid.itertuples():
    team_points_mapping[row.favorite] = (row.over_under - row.line) / 2
    team_points_mapping[row.underdog] = (row.over_under + row.line) / 2

number of teams playing on sunday at 1pm or 4pm: 22


## Split up by position and filter

In [4]:
df_filtered = df_players[
    (df_players["team"].isin(valid_teams))
    & (df_players["salary"] > 0)
    & (df_players["projected_points"] > 0)
    & (df_players["start_sit_score"] >= 2.33)  # start/sit grade >= C+
]

# for each position, filter the list using a set of criteria
df_qb = df_filtered[
    (df_filtered["position"] == "QB")
    & (df_filtered["salary"] >= 5000)
    & (df_filtered["projected_points_per_salary"] > 2.5)
]

df_rb = df_filtered[
    (df_filtered["position"] == "RB")
    & (df_filtered["salary"] >= 4000)
    & (df_filtered["projected_points_per_salary"] > 2.5)
]

df_wr = df_filtered[
    (df_filtered["position"] == "WR")
    & (df_filtered["salary"] >= 4000)
    & (df_filtered["projected_points_per_salary"] > 2.5)
]

df_te = df_filtered[
    (df_filtered["position"] == "TE")
    & (df_filtered["salary"] >= 3000)
    & (df_filtered["projected_points_per_salary"] > 2.0)
]

df_dst = df_filtered[
    (df_filtered["position"] == "DST")
    & (df_filtered["salary"] >= 2000)
    & (df_filtered["projected_points_per_salary"] > 1.5)
]

# combine them all back together
df_players_filtered = pd.concat([df_qb, df_rb, df_wr, df_te, df_dst]).reset_index(
    drop=True
)

print("number of QB:", len(df_qb))
print("number of RB:", len(df_rb))
print("number of WR:", len(df_wr))
print("number of TE:", len(df_te))
print("number of DST:", len(df_dst))
print("total players:", len(df_players_filtered))

number of QB: 12
number of RB: 15
number of WR: 20
number of TE: 8
number of DST: 9
total players: 64


## Define Player and Team classes

In [5]:
class Player(T.NamedTuple):
    name: str
    position: str
    team: str
    opponent: str
    salary: int
    points_per_game: float
    points_per_game_per_salary: float
    expected_team_points: float
    expected_opponent_points: float
    projected_points: float
    start_sit_score: float
    start_sit_grade: str


class Team(T.NamedTuple):
    QB: Player
    RB1: Player
    RB2: Player
    WR1: Player
    WR2: Player
    WR3: Player
    TE: Player
    Flex: Player
    DST: Player

    @property
    def total_salary(self) -> int:
        keys = self.__annotations__.keys()
        salaries = [self.__getattribute__(k).salary for k in keys]
        return sum(salaries)

    @property
    def offense_playing_defense(self) -> bool:
        keys = self.__annotations__.keys()
        offense_teams = [self.__getattribute__(k).team for k in keys if k != "DST"]
        if self.DST.opponent in offense_teams:
            return True
        return False

    @property
    def max_num_players_same_team(self) -> int:
        keys = self.__annotations__.keys()
        teams = [self.__getattribute__(k).team for k in keys]
        return max(Counter(teams).values())

    @property
    def total_ppg_ps(self) -> float:
        keys = self.__annotations__.keys()
        ppg_ps = [self.__getattribute__(k).points_per_game_per_salary for k in keys]
        return round(sum(ppg_ps), 2)

    @property
    def total_ppg(self) -> float:
        keys = self.__annotations__.keys()
        ppg = [self.__getattribute__(k).points_per_game for k in keys]
        return round(sum(ppg), 2)

    @property
    def total_expected_team_points(self) -> float:
        keys = self.__annotations__.keys()
        offense_points = sum(
            [self.__getattribute__(k).expected_team_points for k in keys if k != "DST"]
        )
        points = offense_points - self.DST.expected_opponent_points
        return round(points, 2)
    
    @property
    def total_projected_points(self) -> float:
        keys = self.__annotations__.keys()
        scores = [self.__getattribute__(k).projected_points for k in keys]
        return round(sum(scores), 2)
    
    @property
    def total_start_sit_score(self) -> float:
        keys = self.__annotations__.keys()
        scores = [self.__getattribute__(k).start_sit_score for k in keys]
        return round(sum(scores), 2)
    
    @property
    def avg_start_sit_score(self) -> float:
        keys = self.__annotations__.keys()
        scores = [self.__getattribute__(k).start_sit_score for k in keys]
        return round(sum(scores) / len(scores), 2)

    @property
    def qb_rb_same_team(self) -> bool:
        keys = self.__annotations__.keys()
        rb_teams = [
            self.__getattribute__(k).team
            for k in keys
            if self.__getattribute__(k).position == "RB"
        ]
        if self.QB.team in rb_teams:
            return True
        return False

    @property
    def qb_wr_same_team(self) -> bool:
        keys = self.__annotations__.keys()
        wr_teams = [
            self.__getattribute__(k).team
            for k in keys
            if self.__getattribute__(k).position == "WR"
        ]
        if self.QB.team in wr_teams:
            return True
        return False
    
    @property
    def te_wr_same_team(self) -> bool:
        keys = self.__annotations__.keys()
        wr_teams = [
            self.__getattribute__(k).team
            for k in keys
            if self.__getattribute__(k).position == "WR"
        ]
        if self.TE.team in wr_teams:
            return True
        return False

## Create lists of players and their attributes

In [6]:
# create list of all players using the Player class defined above
all_players = [
    Player(
        name=row.name,
        position=row.position,
        team=row.team,
        opponent=row.opponent,
        salary=row.salary,
        points_per_game=row.points_per_game,
        points_per_game_per_salary=row.points_per_game_per_salary,
        expected_team_points=team_points_mapping[row.team],
        expected_opponent_points=team_points_mapping[row.opponent],
        projected_points=row.projected_points,
        start_sit_score=row.start_sit_score,
        start_sit_grade=row.start_sit_grade,
    )
    for row in df_players_filtered.itertuples()
]

# separate out players by position
qb_players = [p for p in all_players if p.position == "QB"]
wr_players = [p for p in all_players if p.position == "WR"]
rb_players = [p for p in all_players if p.position == "RB"]
te_players = [p for p in all_players if p.position == "TE"]
dst_players = [p for p in all_players if p.position == "DST"]

# get all combinations of 3 WR and 3 RB (assume flex is RB)
# we can change this if we want a team where the flex is also a WR
wr_combinations = list(combinations(wr_players, r=4))
rb_combinations = list(combinations(rb_players, r=2))

# filter combinations to only include one player per team
wr_combinations = [
    c for c in wr_combinations if len(c) == len(set([p.team for p in c]))
]
rb_combinations = [
    c for c in rb_combinations if len(c) == len(set([p.team for p in c]))
]

total_num_combinations = (
    len(wr_combinations)
    * len(rb_combinations)
    * len(qb_players)
    * len(te_players)
    * len(dst_players)
)

print("number of WR combinations:", len(wr_combinations))
print("number of RB combinations:", len(rb_combinations))
print("total number of combinations:", total_num_combinations)

number of WR combinations: 4090
number of RB combinations: 105
total number of combinations: 371044800


## Create list of feasible teams

In [7]:
# randomly downsample the WR/RB combinations
random_wr_combs = random.sample(wr_combinations, k=2000)
random_rb_combs = random.sample(rb_combinations, k=100)

start_time = time.time()
total_count = 0
feasible_teams = []

# loop through each position, create a team, and filter
for qb in qb_players:
    for te in te_players:
        for dst in dst_players:
            for wrs in random_wr_combs:
                for rbs in random_rb_combs:
                    team = Team(
                        QB=qb,
                        RB1=rbs[0],
                        RB2=rbs[1],
                        WR1=wrs[0],
                        WR2=wrs[1],
                        WR3=wrs[2],
                        Flex=wrs[3], # flex can be RB or WR
                        TE=te,
                        DST=dst,
                    )
                    # filter teams that
                    #  1) match the salary constraint;
                    #  2) offenive players not playing defense;
                    #  3) have no more than 2 players on the same team;
                    #  4) QB and RB are not on the same team;
                    #  5) QB and WR are on same team; and
                    #  6) WR and TE are not on the same team.
                    if (
                        team.total_salary == 50000
                        and not team.offense_playing_defense
                        and team.max_num_players_same_team <= 2
                        and not team.qb_rb_same_team
                        and team.qb_wr_same_team
                        and not team.te_wr_same_team
                    ):
                        feasible_teams.append(team)

                    total_count += 1

elapsed_time = time.time() - start_time
print(f"total run time: {round(elapsed_time / 60, 1)} minutes")
print(f"number of feasible teams found: {len(feasible_teams)}")
print(f"percent feasible: {round(len(feasible_teams) / total_count * 100, 1)}%")

total run time: 9.4 minutes
number of feasible teams found: 180340
percent feasible: 0.1%


## Sort and display top team by aggregate score

In [9]:
# sort and score by total ppg
by_total_ppg = sorted(feasible_teams, key=lambda x: x.total_ppg, reverse=True)
team_score_by_total_ppg = {
    team: 1 - i / len(by_total_ppg) for i, team in enumerate(by_total_ppg)
}

# sort and score by total expected team points
by_total_expected_team_points = sorted(
    feasible_teams, key=lambda x: x.total_expected_team_points, reverse=True
)
team_score_by_total_expected_team_points = {
    team: 1 - i / len(by_total_expected_team_points)
    for i, team in enumerate(by_total_expected_team_points)
}

# sort and score by projected points
by_total_projected_points = sorted(
    feasible_teams, key=lambda x: x.total_projected_points, reverse=True
)
team_score_by_total_projected_points = {
    team: 1 - i / len(by_total_projected_points)
    for i, team in enumerate(by_total_projected_points)
}

# sort and score by start/sit score
by_start_sit_score = sorted(
    feasible_teams, key=lambda x: x.avg_start_sit_score, reverse=True
)
team_score_by_start_sit_score = {
    team: 1 - i / len(by_start_sit_score) for i, team in enumerate(by_start_sit_score)
}

# create tuples of teams and weighted average score
team_score = []
for team in feasible_teams:
    score1 = team_score_by_total_ppg[team]
    score2 = team_score_by_total_expected_team_points[team]
    score3 = team_score_by_total_projected_points[team]
    score4 = team_score_by_start_sit_score[team]
    avg_score = (0.1 * score1 + 0.2 * score2 + 0.4 * score3 + 0.3 * score4) / 4
    team_score.append((team, avg_score))

# sort by score
by_score = sorted(team_score, key=lambda x: x[1], reverse=True)[:3]

# print out the details
for i, team_score in enumerate(by_score):
    team = team_score[0]
    print(f"***** team #{i + 1} *****")
    for pos in team.__annotations__.keys():
        p = getattr(team, pos)
        print(
            f"{pos}: {p.name} ({p.team.upper()}) | ${p.salary} | Proj: {p.projected_points} | Grade: {p.start_sit_grade}"
        )
    print(f"total projected points: {team.total_projected_points}")
    print("")

***** team #1 *****
QB: Mahomes II, Patrick (KAN) | $7400 | Proj: 24.0 | Grade: A+
RB1: Jones, Aaron (GB) | $7200 | Proj: 21.0 | Grade: A+
RB2: Johnson, David (HOU) | $5300 | Proj: 15.2 | Grade: B+
WR1: Claypool, Chase (PIT) | $5700 | Proj: 15.5 | Grade: B+
WR2: Hill, Tyreek (KAN) | $6400 | Proj: 18.9 | Grade: A+
WR3: Evans, Mike (TAM) | $6200 | Proj: 16.0 | Grade: A-
TE: Tonyan, Robert (GB) | $4600 | Proj: 11.0 | Grade: B+
Flex: Williams, Mike (LAC) | $4700 | Proj: 13.0 | Grade: C+
DST: Washington (WAS) | $2500 | Proj: 7.0 | Grade: B-
total projected points: 141.6

***** team #2 *****
QB: Mahomes II, Patrick (KAN) | $7400 | Proj: 24.0 | Grade: A+
RB1: Gurley, Todd (ATL) | $6000 | Proj: 17.4 | Grade: A
RB2: Johnson, David (HOU) | $5300 | Proj: 15.2 | Grade: B+
WR1: Ridley, Calvin (ATL) | $7300 | Proj: 19.1 | Grade: A+
WR2: Hill, Tyreek (KAN) | $6400 | Proj: 18.9 | Grade: A+
WR3: McLaurin, Terry (WAS) | $5800 | Proj: 17.1 | Grade: A
TE: Tonyan, Robert (GB) | $4600 | Proj: 11.0 | Grade: 